In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("APLHAVANTAGEAPIKEY")

print(api_key)

In [ ]:
import requests

url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&apikey=" + api_key
response = requests.get(url)
print(response.status_code)
data = response.json()

# Display the latest 5 days of daily prices in a readable format
time_series = data.get("Time Series (Daily)", {})
for date in sorted(time_series.keys(), reverse=True)[:5]:
    print(f"Date: {date}")
    for key, value in time_series[date].items():
        print(f"  {key}: {value}")
    print("-" * 30)


In [ ]:
# I want to store the data in a SQL database. Columns should be: Date, Open, High, Low, Close, Volume.


import sqlite3
def create_table():
    conn = sqlite3.connect('stock_data.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS daily_prices (
            Date TEXT PRIMARY KEY,
            Open REAL,
            High REAL,
            Low REAL,
            Close REAL,
            Volume INTEGER
        )
    ''')
    conn.commit()
    conn.close()    

def insert_data(date, open_price, high_price, low_price, close_price, volume):
    conn = sqlite3.connect('stock_data.db')
    cursor = conn.cursor()
    cursor.execute('''
        INSERT OR REPLACE INTO daily_prices (Date, Open, High, Low, Close, Volume)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (date, open_price, high_price, low_price, close_price, volume))
    conn.commit()
    conn.close()

create_table()

# Insert the data into the database
for date, prices in time_series.items():
    insert_data(
        date,
        float(prices["1. open"]),
        float(prices["2. high"]),
        float(prices["3. low"]),
        float(prices["4. close"]),
        int(prices["5. volume"])
    )

# Query the database to verify the data was inserted correctly
def query_data():
    conn = sqlite3.connect('stock_data.db')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM daily_prices ORDER BY Date DESC LIMIT 5')
    rows = cursor.fetchall()
    conn.close()
    return rows

# Print the last 5 entries from the database
last_entries = query_data()
for entry in last_entries:
    print(f"Date: {entry[0]}, Open: {entry[1]}, High: {entry[2]}, Low: {entry[3]}, Close: {entry[4]}, Volume: {entry[5]}")
# The code above creates a SQLite database, stores daily stock prices, and retrieves the last 5 entries.
# Ensure the database and table are created before inserting data.




In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

url = "https://u.today/search/node?keys=bitcoin"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Extract article titles, dates, authors, and links
articles = soup.find_all('div', class_='news__item')
data = []
seen = set()
for article in articles:
    # Title
    title_tag = article.find('div', class_='news__item-title')
    title = title_tag.get_text(strip=True) if title_tag else 'No Title'
    # Date
    date_tag = article.find('div', class_='humble')
    date = date_tag.get_text(strip=True) if date_tag else 'No Date'
    # Author
    author_tag = article.find('a', class_='humble humble--author')
    author = author_tag.get_text(strip=True) if author_tag else 'Unknown'
    # Link
    link_tag = article.find('a', class_='news__item-body')
    link = link_tag['href'] if link_tag and link_tag.has_attr('href') else 'No Link'

    # Use (title, date, author, link) as a unique identifier
    identifier = (title, date, author, link)
    if identifier not in seen:
        data.append({
            'Title': title,
            'Date': date,
            'Author': author,
            'Link': link
        })
        seen.add(identifier)

# Create a DataFrame from the scraped data
df = pd.DataFrame(data)
df

In [ ]:
# Save the DataFrame to a CSV file
df.to_csv('articles.csv', index=False)

## Streamlit

In [ ]:
import streamlit as st

st.title("Hello World")
